In [ ]:
%matplotlib inline
import matplotlib
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import math

input_text_file = "../data/Fuzuli-1.txt"

with open(input_text_file, "r", encoding="utf-8") as f:
    input_text = f.read()

# Whitespace Tokenization

In [ ]:
# Whitespace tokenization - letter only, stored in a dataframe
tokens = re.findall(r"\p{L}+(?:-\p{L}+)*", input_text)
df = pd.DataFrame(tokens, columns=["tokens"])
df['tokens'] = df['tokens'].str.lower()

In [ ]:
# Unique tokens with counts and percentage
counts = df['tokens'].value_counts()
total = counts.sum()
unique_df = pd.DataFrame({
    'token': counts.index,
    'count': counts.values,
    'frequency (%)': (counts.values/total*100).round(2)
})

# Total number of tokens
print("Token number of tokens:", total)

# Number of unique tokens
unique_count = df['tokens'].nunique() 
print("\nNumber of unique tokens:", unique_count)

# Top 15 most frequent tokens
print("\n",unique_df.head(15).to_string(index=False))

In [ ]:
top15 = unique_df.head(15)

plt.figure(figsize=(10,6))
plt.bar(top15['token'], top15['count'], color="red")

plt.xlabel("Token")
plt.ylabel("Frequency")
plt.title("Top 15 Most Frequent Tokens")
plt.xticks(rotation=45)
plt.show()


# Heaps' Law

In [ ]:
# Heaps Law
df_clean = df[~df['tokens'].isin(['.', '!', '?'])].copy()

def get_N_V(df):
    
    N, V = [], []
    all_tokens = df['tokens'].tolist()
    seen = set()

    for i, tok in enumerate(all_tokens, start=1):
        N.append(i)
        seen.add(tok)
        V.append(len(seen))

    return N, V

# linear regression
def lin_reg(N, V):

    x = [math.log(n) for n in N]
    y = [math.log(v) for v in V]

    x_mean = sum(x) / len(x)
    y_mean = sum(y) / len(y)

    # y = a + beta*x | Least Square Estimation
    beta = sum((xi - x_mean)*(yi - y_mean) for xi, yi in zip(x,y))/sum((xi - x_mean)**2 for xi in x)
    a = y_mean - beta * x_mean
    k = math.exp(a)

    return k, beta

N, V = get_N_V(df_clean)

k, beta = lin_reg(N, V)

print("Value of k:", k, "\nValue of beta:", beta)

In [ ]:
import matplotlib.pyplot as plt

# Observed values
plt.figure(figsize=(8,6))
plt.plot(N, V, label="Observed Vocabulary Growth", color="blue")

# Predicted values using Heaps' law
V_pred = [k * (n ** beta) for n in N]
plt.plot(N, V_pred, label=f"Heaps' Law Fit (k={k:.2f}, β={beta:.2f})", color="red", linestyle="--")

# Labels and legend
plt.xlabel("Total Number of Tokens (N)")
plt.ylabel("Vocabulary (V)")
plt.title("Vocabulary Growth vs. Total Tokens")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
corpus = [list(token) for token in df['tokens']]

In [ ]:
def sentence_segmentation(text):
    # split on ., !, ? only when followed by space or end of string
    sentence_endings = re.compile(
        r'([.!?])([\'")\]]*)(?=\s|$)'  # punctuation + optional quotes/brackets, followed by space or end
    )
    # Delimiter for splitting
    segmented = sentence_endings.sub(r'\1\2</s>', text)
    sentences = [s.strip() for s in segmented.split('</s>') if s.strip()]
    return sentences


# BPE Algorithm

In [ ]:
def learn_bpe(corpus, num_merges):
    merges = []
    merge_freqs = []   # store (pair, frequency) at each step
    for _ in range(num_merges):
        # Count pairs fresh each iteration
        vocab = {}
        for word in corpus:
            for i in range(len(word)-1):
                pair = (word[i], word[i+1])
                vocab[pair] = vocab.get(pair, 0) + 1
        
        if not vocab:
            break
        
        # Most frequent pair
        most_frequent = max(vocab, key=vocab.get)
        merges.append(most_frequent)
        merge_freqs.append((most_frequent, vocab[most_frequent]))
        
        # Merge in corpus
        new_symbol = ''.join(most_frequent)
        new_corpus = []
        for word in corpus:
            i = 0
            new_word = []
            while i < len(word):
                if i < len(word)-1 and (word[i], word[i+1]) == most_frequent:
                    new_word.append(new_symbol)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_corpus.append(new_word)
        corpus = new_corpus
    
    return merges, corpus, merge_freqs


# Run BPE
merges, updated_corpus, merge_freqs = learn_bpe(corpus, num_merges=1000)

all_tokens = [tok for word in updated_corpus for tok in word]

longest_token = max(all_tokens, key=len)

print("Longest token:", longest_token, "with length:", len(longest_token))

In [ ]:
# Take top 15 by frequency
top15 = sorted(merge_freqs, key=lambda x: x[1], reverse=True)[:15]

# Print results
print("Top 15 frequent BPE pairs:")
for pair, freq in top15:
    print(f"{pair}: {freq}")

# Plot histogram
labels = [f"{a}{b}" for (a,b), _ in top15]
counts = [freq for _, freq in top15]

plt.figure(figsize=(10,6))
plt.bar(labels, counts, color="red")
plt.xlabel("BPE Tokens")
plt.ylabel("Frequency")
plt.title("Top 15 Frequent BPE Tokens")
plt.show()


# Processing Text With BPE Tokenizer

In [ ]:
def apply_bpe(text, merges):
    text = text.lower()
    # Represent word as characters + end-of-word marker
    chars = list(text) + ["</w>"]
    
    # Apply merges in the order they were learned
    for merge in merges:
        merged = ''.join(merge)
        new_chars = []
        i = 0
        while i < len(chars):
            if i < len(chars)-1 and (chars[i], chars[i+1]) == merge:
                new_chars.append(merged)
                i += 2
            else:
                new_chars.append(chars[i])
                i += 1
        chars = new_chars
    
    return ' '.join(chars)

In [ ]:
sample_text = "Bakıdan əlimə çatan qadın jurnallarının birində Kələkoş sözü diqqətimi cəlb etdi. " \
"Bu, elə reseptini axtardığım Kələkoş idi! Amma bu dəfı adın yanında resept də var idi! " \
"Beləliklə sirr dolu Kələkoşu bişirmək fürsətini əldə etmiş oldum. Çox ləzzətli şorbadır! " \
"Tez-tez menyumuza qolaq olur. Buyurun, siz de dadına baxın."

# Sample text - Sentence Segmentation + BPE tokenization

In [ ]:
def count_tokens(bpe_text):
    return len(bpe_text.split())


def sentence_tokenizer(sample_text):
    segmented_sentences = sentence_segmentation(sample_text)

    total_tokens = 0
    i = 0

    for sentence in segmented_sentences:
        i+=1
        res = apply_bpe(sentence, merges)
        token_count = count_tokens(res)
        total_tokens += token_count
        print(f"{i})", res, f"{token_count} tokens", "\n")
    print("Total number of tokens:", total_tokens)

sentence_tokenizer(sample_text)

# Input Sentence Segmentation + BPE Tokenization

In [ ]:
while True:
    sample_text = input("Enter a sentence (or press Enter to quit): ")
    if not sample_text.strip():
        break
    sentence_tokenizer(sample_text)

# Spell Checking

In [ ]:
import LevensteinBK 
import unicodedata

def strip_combining(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
with open("Fuzuli-I.txt", "r", encoding="utf-8") as infile:
    content = infile.read()

content = content.lower()
Tokens = re.findall(r"\b[^\W\d_]+(?:[\'’\-][^\W\d_]+)*\b", content)
Tokens = [t.replace("’", "'") for t in Tokens]
Tokens = [strip_combining(t) for t in Tokens]
vocabulary = set()

for t in Tokens:
    vocabulary.add(t)


tt = LevensteinBK.BKtree(vocabulary)
search_word = "lə'li-nabın"
res = tt.search(search_word, 2)
sorted_res = sorted(res, key = lambda w:( LevensteinBK.Levenstein(w.label, search_word)))

for i in sorted_res:
    print(i)

